# Rolling 30 Days Reactovation Rate
### of Resellers in a dropshipping company

In a drpshipping company, once a reseller makes an order, she/he is considered retained for the next month, i.e. 29 days. If the place order in the course of this 29 days, they are retained for the next month. If they don't place order within 29 days, they are churned. Some resellers reactivate some days after churn and start placing orders over. 

With this report I created a chart showing the rolling reactivation rate of resellers over 30 days. Here is my approach to create this report;
1. For each reseller, calculate the number of days beteween consequent orders (DistanceFromPreviousOrder).
2. If DistanceFromPreviousOrder >= 30, then Reactivated = 1, else Reactivated = 0.
3. For each date, count the number of Reactivated = 1 in interval [date-29 days, date].
4. Divide the reactivation of each day by all churned resellers to obtain the reactivation rate.

To calculate the number of churned resellers, I found the date of their last order and calculated the difference between current_date and last_order_date. If the difference is larger than 30, then the reseller has churned.

WITH basic1 AS ( SELECT
      orders.ID AS OrderID,
      orders.UserID AS UserID,
      -- `orders`.`CreatedAt` AS `OrderDate`,
      cast(`orders`.`CreatedAt` AS DATE) AS `OrderDate`,
      `orders`.`FakeDelete` AS `FakeDelete`,
      `orders`.`Type` AS `Type`,
      --  users.CreatedAt as joinDate,
      cast(users.CreatedAt AS DATE) AS `JoinDate`,
      `Vw Orders Is Net`.`isGross` AS `isGross`,
      `Vw Orders Is Net`.`isNet` AS `isNet`,
      ROW_NUMBER() OVER ( PARTITION BY UserID ORDER BY orders.CreatedAt DESC ) AS OrderOrder,
      DATEDIFF( `orders`.`CreatedAt`, coalesce( LEAD(`orders`.`CreatedAt`) OVER ( PARTITION BY UserID ORDER BY UserID), 0))         AS DistanceFromPreviousOrder
    FROM
      `orders`
      INNER JOIN `users` ON `orders`.`UserID` = `users`.`ID`
      INNER JOIN `vw_orders_is_net` AS `Vw Orders Is Net` ON `orders`.`ID` = `Vw Orders Is Net`.`OrderID`
    WHERE
      (`orders`.`Type` = 'order')
      AND (`Vw Orders Is Net`.`isGross` = 1) ),
  
basic2 AS ( SELECT *,
    CASE
        WHEN `DistanceFromPreviousOrder` >= 30 THEN 1
        ELSE 0
      END AS `Reactivated`,
      DATE_ADD(`OrderDate`, INTERVAL -29 DAY) AS `StartOfPeriod`,
      `OrderDate` AS `EndOfPeriod`
    FROM
      basic1 ),
  
basic3 AS ( SELECT
      OrderDate,
      sum(Reactivated) AS Reactivated
    FROM
      basic2
    GROUP BY
      OrderDate
    ORDER BY
      OrderDate ASC ),
  
churn AS ( SELECT
      count(*) allTimeChurn
    FROM
      basic1
    WHERE
      OrderOrder = 1
      AND DATEDIFF(current_date(), OrderDate) > 30 )
  
  
SELECT
  OrderDate,
  coalesce( SUM(Reactivated) OVER ( ORDER BY OrderDate ROWS BETWEEN 29 PRECEDING AND CURRENT ROW), 0) / allTimeChurn AS    L30DReactivated
FROM
  basic3,
  churn
ORDER BY
  OrderDate DESC